### *Clean data* and then compare types and tokens before and after pre-processing

* now we just clean the data

In [5]:
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import it_core_news_sm
nlp = it_core_news_sm.load()

In [31]:
df = pd.read_excel('uncleaned_data.xlsx', encoding='utf8')

In [32]:
df.head()

,Unnamed: 0,userid,username,tweet,nretweets,party,location,totaltweets,followers
0,0,2752322571,Carlo Bravi,"https://t.co/s1wks2fc6K\n\n""Anch’io mi sono af...",0,pdnetwork,"Torino, Piemonte",5522,218
1,1,3124013698,Artemisia Gentileschi,"#Coronavirus, perché in #Lombardia mancano le ...",0,pdnetwork,NaN,36214,127
2,2,2869471262,Mario Ruggieri,Attivo oggi il Covid2 Hospital al Columbus Gem...,183,pdnetwork,NaN,11605,118
3,3,514782781,Katia mondin,#Coronavirus: appello della #Cri: Non abbandon...,57,pdnetwork,NaN,5471,40
4,4,429139349,canzio dusi,"📌 #Coronavirus, l'evoluzione del numero di cas...",20,pdnetwork,osnago,42303,2095


In [33]:
del df['Unnamed: 0']

In [34]:
df.head()

,userid,username,tweet,nretweets,party,location,totaltweets,followers
0,2752322571,Carlo Bravi,"https://t.co/s1wks2fc6K\n\n""Anch’io mi sono af...",0,pdnetwork,"Torino, Piemonte",5522,218
1,3124013698,Artemisia Gentileschi,"#Coronavirus, perché in #Lombardia mancano le ...",0,pdnetwork,NaN,36214,127
2,2869471262,Mario Ruggieri,Attivo oggi il Covid2 Hospital al Columbus Gem...,183,pdnetwork,NaN,11605,118
3,514782781,Katia mondin,#Coronavirus: appello della #Cri: Non abbandon...,57,pdnetwork,NaN,5471,40
4,429139349,canzio dusi,"📌 #Coronavirus, l'evoluzione del numero di cas...",20,pdnetwork,osnago,42303,2095


In [35]:
# no tags, links nor '\n' 
df['tweet'] = df['tweet'].str.replace(r"(@[A-Za-z0-9_]+)|(\w+:\/\/\S+)|([\n]+)",'', regex=True)
df['tweet'] = df['tweet'].str.replace("#",'') #i keep the text following the # though because i want to keep track of them

df['tweet'] = df['tweet'].apply(lambda x: x.lower()) #lower case
df['tweet'] = df['tweet'].str.replace(r"[^\w\s]",' ', regex=True) # remove punctuation
df['tweet'] = df['tweet'].str.replace(r"[0-9]+", '', regex=True) # no numbers
df['tweet'] = df['tweet'].str.replace(r"\s[\s]+", ' ', regex=True) # no more than two white spaces
df['tweet'] = df['tweet'].str.replace(r"^[\s]+", '', regex=True) #no white spaces at the beginning


In [41]:
df.head()

,userid,username,tweet,nretweets,party,location,totaltweets,followers
0,2752322571,Carlo Bravi,anch io mi sono affacciato sul balcone e ho fr...,0,pdnetwork,"Torino, Piemonte",5522,218
1,3124013698,Artemisia Gentileschi,coronavirus perché in lombardia mancano le mas...,0,pdnetwork,NaN,36214,127
2,2869471262,Mario Ruggieri,attivo oggi il covid hospital al columbus geme...,183,pdnetwork,NaN,11605,118
3,514782781,Katia mondin,coronavirus appello della cri non abbandonate ...,57,pdnetwork,NaN,5471,40
4,429139349,canzio dusi,coronavirus l evoluzione del numero di casi to...,20,pdnetwork,osnago,42303,2095


In [42]:
stops = pd.read_csv('stopwords-it.txt', ' ') # I added some words to this list in order to get rid of as much variability as 
                                             # possible in the data
stops = list(stops['stopwords'])

In [43]:
documents = list(df['tweet'].values)
documents

['anch io mi sono affacciato sul balcone e ho fraternizzato con i vicini di pianerottolo che finora salutavo solo in ascensore sento dire che dopo il coronavirus nulla sarà più come prima non ci credo ma se anche fosse non è detto che sia un male ',
 'coronavirus perché in lombardia mancano le mascherine protettive fontana e gallera non raccontano tutta la storia ',
 'attivo oggi il covid hospital al columbus gemelli una risposta eccezionale in pochi giorni grazie a tutti gli operatori che sono in prima linea un altro passo concreto nella costruzione della rete contro il coronavirus ora avanti con l apertura del covid e covid hospital ',
 'coronavirus appello della cri non abbandonate gli animali e una fake news che cani e gatti contagino gli esseri umani videoansa ',
 'coronavirus l evoluzione del numero di casi totali in italia dal febbraio al marzocoviditaly covid ',
 'le squadre di calcio inglesi ripiegano su fifa e footballmanager dopo lo stop forzato a causa del coronavirus ',
 '

In [ ]:
# CREATE MACRO_PARTY AND THEN STATUS

In [19]:
from collections import defaultdict
pulizia = defaultdict(list)

for row in df.iterrows(): #the .iterrows iterates over the pair (index, Series)
    label = row[1].party #thus, we specify [1] to get the Series itself not the index and then we access the column score
    text = row[1].userid # and the column text
    pulizia[text].append(label)
    


In [18]:
users = df['userid'].unique().tolist()

In [47]:
df['macro_party'] = 'Indifferent'

In [48]:
for user in users:
    if pulizia[user] == ['CasaPoundItalia']:
        df['macro_party'][df['userid']==user] = 'EstremaDestra'
    if pulizia[user] == ['Mov5Stelle']:
        df['macro_party'][df['userid']==user] = 'Centro'
    if 'FratellidItalia' in pulizia[user] and 'forza_italia' in pulizia[user] and not 'pdnetwork' in pulizia[user] and not 'liberi_uguali' in pulizia[user]:
        df['macro_party'][df['userid']==user] = 'CentroDestra'
    if not 'LegaSalvini' in pulizia[user] and not 'FratellidItalia' in pulizia[user] and not 'forza_italia' in pulizia[user] and not 'CasaPoundItalia' in pulizia[user] and 'pdnetwork' in pulizia[user]:
        df['macro_party'][df['userid']==user] = 'CentroSinistra'
    if not 'LegaSalvini' in pulizia[user] and not 'FratellidItalia' in pulizia[user] and not 'forza_italia' in pulizia[user] and not 'CasaPoundItalia' in pulizia[user] and 'liberi_uguali' in pulizia[user]:
        df['macro_party'][df['userid']==user] = 'CentroSinistra'
    if 'LegaSalvini' in pulizia[user] and not 'FratellidItalia' in pulizia[user] and not 'forza_italia' in pulizia[user] and not 'CasaPoundItalia' in pulizia[user] and not 'pdnetwork' in pulizia[user] and not 'liberi_uguali' in pulizia[user]:
        df['macro_party'][df['userid']==user] = 'LegaSalvini'


In [ ]:
for user in users:
    if pulizia[user] == ['pdnetwork']:
        df['macro_party'][df['userid']==user] = 'CentroSinistra'
    
    if pulizia[user] == ['forza_italia']:
        df['macro_party'][df['userid']==user] = 'CentroDestra'
    
    if pulizia[user] == ['FratellidItalia']:
        df['macro_party'][df['userid']==user] = 'CentroDestra'
        
    if 'FratellidItalia' in pulizia[user] and 'Mov5Stelle' in pulizia[user]:
        df['macro_party'][df['userid']==user] = 'CentroDestra'

In [49]:
df.macro_party.unique()

array(['CentroSinistra', 'Indifferent', 'CentroDestra', 'Centro',
       'LegaSalvini', 'EstremaDestra'], dtype=object)

In [55]:
len(df[df['macro_party']=='Indifferent'])

129700

In [56]:
df.head()

,userid,username,tweet,nretweets,party,location,totaltweets,followers,macro_party
0,2752322571,Carlo Bravi,anch io mi sono affacciato sul balcone e ho fr...,0,pdnetwork,"Torino, Piemonte",5522,218,CentroSinistra
1,3124013698,Artemisia Gentileschi,coronavirus perché in lombardia mancano le mas...,0,pdnetwork,NaN,36214,127,Indifferent
2,2869471262,Mario Ruggieri,attivo oggi il covid hospital al columbus geme...,183,pdnetwork,NaN,11605,118,CentroSinistra
3,514782781,Katia mondin,coronavirus appello della cri non abbandonate ...,57,pdnetwork,NaN,5471,40,CentroSinistra
4,429139349,canzio dusi,coronavirus l evoluzione del numero di casi to...,20,pdnetwork,osnago,42303,2095,Indifferent


In [58]:
df.shape

(153653, 9)

In [59]:
df2 = df.groupby(by=['userid']).apply(lambda x: x.drop_duplicates(subset=['tweet']))

In [60]:
df2.shape

(41200, 9)

In [61]:
df2.tweet.nunique()

15488

In [62]:
pd.DataFrame.to_excel(df2,"cleaned_data.xlsx",'data')

* **now I apply different pre-processing techniques to evaluate the one with the best performance** 
In the final file where I conducted the analysis, I deleted the columns with lemmatization and stemming. 

In [ ]:
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import it_core_news_sm
nlp = it_core_news_sm.load()

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('cleaned_data.xlsx', encoding='utf8')

In [4]:
df.head()

,userid,username,tweet,nretweets,party,location,totaltweets,followers,macro_party
0,16603,gillo,la mia eroina in questo periodo di italylockdo...,0,pdnetwork,NaN,10491,1299,Indifferent
1,536013,Mirco Di Marcello,zeppole al tempo del coronavirus fatte con que...,1,liberi_uguali,"Alba Adriatica, Abruzzo",11465,3417,CentroSinistra
2,613673,IOOOTA S.r.l.,lo sappiamo è un momento difficile per tutti c...,0,FratellidItalia,"Bologna, Italy",842,1142,Indifferent
3,647043,Andrea Romoli,oggi pomeriggio dalle interverrò su centrosuon...,11,forza_italia,"Milano, Italy",49778,8435,Indifferent
4,647043,Andrea Romoli,che fate stasera per la nottebiancatw noi game...,6,FratellidItalia,"Milano, Italy",49841,8445,Indifferent


In [65]:
df.info() # the only missing data we have are in the location. However, here I focus on the political parties so I do not care

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41200 entries, 0 to 41199
Data columns (total 9 columns):
userid         41200 non-null object
username       41200 non-null object
tweet          41200 non-null object
nretweets      41200 non-null object
party          41200 non-null object
location       24014 non-null object
totaltweets    41200 non-null object
followers      41200 non-null object
macro_party    41200 non-null object
dtypes: object(9)
memory usage: 2.8+ MB


In [52]:
stops = pd.read_csv('stopwords-it.txt', ' ') # I added some words to this list in order to get rid of as much variability as 
                                             # possible in the data
stops = list(stops['stopwords'])

In [51]:
documents = list(df['tweet'].values)
documents

['la mia eroina in questo periodo di italylockdown è la ragazza che utilizza il linguaggio dei segni durante le conferenze stampa giornaliere della protezione civile covid',
 'zeppole al tempo del coronavirus fatte con quel che avevo fragole e non ciliegie crema senza uova con questo simbolo del momento che stiamo vivendo voglio fare gli auguri a tutti i papà in special modo a quelli che stanno combattendo in corsia ',
 'lo sappiamo è un momento difficile per tutti come tutta italia sta continuando a lavoraredaremoto con tutto il suo team sui vostri progetti il cda di oggi in videoconferenza è stato un nuovo bell esempio di smartworking possibile iorestoacasa ',
 'oggi pomeriggio dalle interverrò su centrosuonosport per uno speciale sull emergenza sanitaria del coronavirus e sulle misure in merito al decreto curaitalia miliardi di euro per medici famiglie aziende iorestoacasa solidarietàdigitale distantimauniti ',
 'che fate stasera per la nottebiancatw noi gamersagainstcovid vogliamo 

In [53]:
lemmas = [[token.lemma_ for token in nlp(sentence) 
            if token.pos_ in {'NOUN', 'VERB', 'PROPN', 'ADJ', 'ADV'} and not token.text in stops] 
           for sentence in documents]

In [54]:
tokens = [[token.text for token in nlp(sentence) if not token.text in stops] for sentence in documents]

In [55]:
from nltk import SnowballStemmer
stemmer = SnowballStemmer('italian')


tweets_stem = [[stemmer.stem(token) for token in tweets] for tweets in tokens]

In [56]:
from collections import defaultdict

stem_origins = defaultdict(lambda: defaultdict(int)) # is a dictionary in the form 
                                                     # {stem : defaultdict(int, {token1 : occurence, token2 : occurence, ..})}

for sentence in tokens:
    for token in sentence:
        stem = stemmer.stem(token)
        stem_origins[stem][token.lower()] += 1

#stem_origins['sal']

In [57]:
tweets_stem2 = [[sorted(stem_origins[token].items(), key=lambda item: item[1])[-1][0] for token in tweet] for tweet in tweets_stem]

In [59]:
df.loc[:, 'lemmas'] = lemmas
df.loc[:, 'STEMS'] = tweets_stem
df.loc[:, 'STEMS2'] = tweets_stem2

In [60]:
df.loc[:,'lemmas'] = df.apply(lambda row: ' '.join(row.lemmas), axis = 1)
df.loc[:,'STEMS'] = df.apply(lambda row: ' '.join(row.STEMS), axis = 1)
df.loc[:,'STEMS2'] = df.apply(lambda row: ' '.join(row.STEMS2), axis = 1)

In [61]:
df['STEMS2'] = df['STEMS2'].str.replace("iorestocasa",'iorestoacasa')
df['STEMS2'] = df['STEMS2'].str.replace("italy",'italia')

In [65]:
df.head()

,userid,username,tweet,nretweets,party,location,totaltweets,followers,macro_party,lemmas,STEMS,STEMS2
0,16603,gillo,la mia eroina in questo periodo di italylockdo...,0,pdnetwork,NaN,10491,1299,CentroSinistra,eroina periodare italylockdown ragazza utilizz...,eroin period italylockdown ragazz utilizz ling...,eroina periodo italialockdown ragazza utilizza...
1,536013,Mirco Di Marcello,zeppole al tempo del coronavirus fatte con que...,1,liberi_uguali,"Alba Adriatica, Abruzzo",11465,3417,CentroSinistra,zeppole fare fragola ciliegia cremare uovo sim...,zeppol fatt fragol cilieg crem uov simbol viv ...,zeppole fatti fragole ciliegi crema uova simbo...
2,613673,IOOOTA S.r.l.,lo sappiamo è un momento difficile per tutti c...,0,FratellidItalia,"Bologna, Italy",842,1142,CentroDestra,sapere difficile italia continuare lavoraredar...,sapp difficil ital continu lavoraredaremot tea...,sappiate difficile italia continua lavoraredar...
3,647043,Andrea Romoli,oggi pomeriggio dalle interverrò su centrosuon...,11,forza_italia,"Milano, Italy",49778,8435,Indifferent,pomeriggio intervenire centrosuonosport specia...,pomerigg interverr centrosuonosport special em...,pomeriggio interverrò centrosuonosport special...
4,647043,Andrea Romoli,che fate stasera per la nottebiancatw noi game...,6,FratellidItalia,"Milano, Italy",49841,8445,Indifferent,fata stasera nottebiancatw gamersagainstcovid ...,fat staser nottebiancatw gamersagainstcovid vo...,fate stasera nottebiancatw gamersagainstcovid ...


In [67]:
pd.DataFrame.to_excel(df,"data_coronavirus.xlsx",'data')